In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

from experiment_utils import *
from src import *


# SAVE TO RESULTS FILE

In [6]:
FOLDER = "RQ4_CONSISTENCY"

if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)

In [7]:
def f(df, reader): # Keep only K and KP
    df = df[(df["ML method"].str.contains('nn'))]
    df['TEMP'] = df['other'].apply(reader._get_losses_used)
    df = df[(~df["bias mitigation"].str.contains('FYP VAE')) | df["TEMP"].str.contains('Sensitive KL loss') ]
    df = df[~(df["TEMP"].str.contains("Flipped"))]
    df = df[~(df["TEMP"].str.contains("Latent"))]
    return df.drop(columns=['TEMP'])

### Per metric count of instances metric was increased from baseline

In [8]:
for attr in [["sex"],["race"],["race", "sex"]]:
    files = [os.path.join("results","RESULTS_"+'_'.join(attr)+".csv")]
    reader = ResultsReader(files)
    reader.change_other_to_losses()

    rel_df = reader.get_relative_metrics()

    #metrics_to_use = ["f1", "MCC", "A_", "DI_FM", "SPD", "SF", "DF"]
    #metrics_to_display = [m for m in reader.metrics if metric_has_substring(m,metrics_to_use )]
    metrics_to_display = reader.metrics
    
    for data in ["Adult", "Compas", "Default"]:           
        df = f(rel_df[(rel_df["data"].str.contains(data))], reader)
        
        if df.shape[0] > 0: # in case some dataset has no results for some metrics
            counts_total = df[(df["bias mitigation"].str.contains('No'))].groupby(["bias mitigation", "other"]).count()[metrics_to_display]
            
            result = df.groupby(["bias mitigation", "other"])[metrics_to_display].apply(lambda x: (x >= 0).sum()).reset_index()
            results = pd.concat([result, counts_total])
            
            results.to_csv(os.path.join(FOLDER ,'PER_METRIC_nr_incr_'+data+'_'+"_".join(attr)+'.csv'), index=False)

File 'results\RESULTS_sex.csv' successfully loaded as DataFrame.
File 'results\RESULTS_race.csv' successfully loaded as DataFrame.
File 'results\RESULTS_race_sex.csv' successfully loaded as DataFrame.
